Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

In [1]:
from bs4 import BeautifulSoup as bs  
from bs4 import BeautifulStoneSoup    
import bs4     
import requests
import json
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import ast

In [2]:
client = MongoClient('localhost', 27017)
db = client['HH_database']
collection = db.HH_API_collection

In [8]:
a = db.collection.find( {} )

In [9]:
for doc in a:
    print(doc)
    print('**************************')
    break

{'_id': ObjectId('616468d5d743c782d93ff30b'), 'id': '48434078', 'premium': False, 'name': 'Data Scientist', 'department': None, 'has_test': False, 'response_letter_required': False, 'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'}, 'salary': {'from': 180000, 'to': 250000, 'currency': 'RUR', 'gross': False}, 'type': {'id': 'open', 'name': 'Открытая'}, 'address': None, 'response_url': None, 'sort_point_distance': None, 'published_at': '2021-10-10T12:06:52+0300', 'created_at': '2021-10-10T12:06:52+0300', 'archived': False, 'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=48434078', 'insider_interview': None, 'url': 'https://api.hh.ru/vacancies/48434078?host=hh.ru', 'alternate_url': 'https://hh.ru/vacancy/48434078', 'relations': [], 'employer': {'id': '1664868', 'name': 'CORE', 'url': 'https://api.hh.ru/employers/1664868', 'alternate_url': 'https://hh.ru/employer/1664868', 'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/

Интересующая сумма

In [10]:
TARGET = 160000

In [26]:
list(db.collection.find( { '$or': [ { 'salary_from': { '$gte': TARGET } }, { 'salary_to': { '$gte': TARGET } } ] }, 
                   {'name': 1, 'area': 1, 'vacancies_url':1, 'employer':1, '_id':0, 'salary_to':1, 'salary_from':1} ))[:2]

[{'name': 'Data Scientist',
  'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
  'employer': {'id': '1664868',
   'name': 'CORE',
   'url': 'https://api.hh.ru/employers/1664868',
   'alternate_url': 'https://hh.ru/employer/1664868',
   'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/589183.png',
    '240': 'https://hhcdn.ru/employer-logo/2797988.png',
    '90': 'https://hhcdn.ru/employer-logo/2797987.png'},
   'vacancies_url': 'https://api.hh.ru/vacancies?employer_id=1664868',
   'trusted': True},
  'salary_from': 180000.0,
  'salary_to': 250000.0},
 {'name': 'Data Scientist',
  'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
  'employer': {'id': '250',
   'name': 'COMTEK Inc.',
   'url': 'https://api.hh.ru/employers/250',
   'alternate_url': 'https://hh.ru/employer/250',
   'logo_urls': {'original': 'https://hhcdn.ru/employer-logo-original/462338.png',
    '90': 'https://hhcdn.ru/employer-logo/2291212.png',
    '2

In [21]:
a = db.collection.find( { '$or': [ { 'salary_from': { '$gte': TARGET } }, { 'salary_to': { '$gte': TARGET } } ] }, 
                   {'name': 1, 'area': 1, 'vacancies_url':1, 'employer':1, '_id':0, 'salary_to':1, 'salary_from':1} )

In [22]:
my_df = pd.DataFrame.from_dict(a, orient='columns')

In [23]:
my_df

,name,area,employer,salary_from,salary_to
0,Data Scientist,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '1664868', 'name': 'CORE', 'url': 'http...",180000.0,250000.0
1,Data Scientist,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '250', 'name': 'COMTEK Inc.', 'url': 'h...",150000.0,250000.0
2,Data scientist,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '3306967', 'name': 'Mind&Machine', 'url...",180000.0,300000.0
3,Data Scientist,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '5488399', 'name': 'Самокат (ООО Умное ...",200000.0,NaN
4,Data Scientist,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '2324020', 'name': 'Точка', 'url': 'htt...",NaN,300000.0
...,...,...,...,...,...
114,"Системный аналитик (ETL, Big-Data, ML)","{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '124', 'name': 'Coleman Services', 'url...",200000.0,NaN
115,Data-аналитик в дивизион «Массовая персонализа...,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '3529', 'name': 'Сбербанк', 'url': 'htt...",NaN,300000.0
116,Data Engineer в платформу Data Lake,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '4023', 'name': 'Райффайзенбанк', 'url'...",300000.0,NaN
117,Data Engineer,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'id': '1319166', 'name': 'Экспириенс', 'url':...",200000.0,NaN
